<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/notebooks/05.07-Support-Vector-Machines_BME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# サポートベクタマシン (Support Vector Machine, SVM)  
  
[In-Depth: Support Vector Machines](https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html), J. VanderPlasより

Support Vector Machines (SMV)は分類と予測で用いられる有名なアルゴリズムである．  
この例ではscikit-learnのSVMを使用する．  
まず必要なライブラリをインポートする．

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## 序論

まずは単純に直線や曲線で分類する．例として，2つのクラスの点が十分に離れている単純なケースを考える．  
データはsklearnで生成する．

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

1つの線(識別境界)で2つのデータセットを分類することができるが，そのような線は無数にあることが分かる．  
いずれにせよ，識別境界が決まったら，新しいデータ(ここでは×)について分類しラベルを与えることができる．
  
しかし，これで終わりではなく，もうちょっと深く考えてみる．

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5);

## マージンを最大化する

SVMは，これを改善するために，異なるクラス間の距離（マージン）を最大化させる．  
マージンとは下の図の灰色の領域となる．これが一番広いものがベストとなる．

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='lightgray', alpha=0.5)

plt.xlim(-1, 3.5);

### フィッティング

Let's see the result of an actual fit to this data: we will use Scikit-Learn's support vector classifier (`SVC`) to train an SVM model on this data.
For the time being, we will use a linear kernel and set the ``C`` parameter to a very large number (we'll discuss the meaning of these in more depth momentarily):

実際にSVMを適用してみる．

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)

    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])

    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, edgecolors='black',
                   facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

求めた識別境界が表示されたはずである．いくつかの(3つ？)のデータはちょうどマージンに接していることがわかる(黒丸で囲った点)．これらを「サポートベクトル」とよぶ．  
  
サポートベクトルの座標を表示してみる．

In [ ]:
model.support_vectors_

このSVMの特徴は，マージンはサポートベクトル「のみ」によって決まることにある．マージンを超えない限り，データの数は重要ではない．  
次の図ではデータの最初の60個と120個から学習したものであるが，3つのサポートベクトルが同じである．

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)

    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

次の例ではスライダーを操作することによって任意のデータ数から得られたモデルを表示することができる．

In [ ]:
from ipywidgets import interact, fixed
interact(plot_svm, N=(10, 200), ax=fixed(None));

### カーネルSVM

分離境界が直線である線形SVMが必ずしもうまくいかない場合，「カーネルSVM」という方法がある．  
実際に線形分離ができない例を見る．

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

この例はどのような直線の分離境界でもデータを分離することができない．しかし，「カーネル関数(核関数)」というものを使い，2次元のデータを3次元上に投影する(次元拡張)ことで，直線で分離することが可能となる．  
  
  次の例では，カーネル関数として放射基底関数$r=e^{-X^2}$を用いている．これは中央が円状に盛り上がった関数である．

In [ ]:
r = np.exp(-(X ** 2).sum(1))

三次元グラフにするとカーネル関数がどのように適用されたかが分かりやすい．

In [ ]:
from mpl_toolkits import mplot3d

ax = plt.subplot(projection='3d')
ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
ax.view_init(elev=20, azim=30)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('r');

もう明らかであるが，高さ方向(r軸)に分離「平面」を置くことで，データが分離できる．  
  
しかし，問題はカーネル関数をどのように決めるかである．まず，放射基底関数の中心が正しい位置にある必要がある．また，そもそも単純な放射基底関数でうまくいくとは限らない．  
  
「カーネルSVM」とは，この問題を解決するため，いったんすべてのデータを中心とする放射基底関数を設けて計算し，その中でアルゴリズムに選択させる方法である．

カーネルSVMを適用し結果を見てみる．今回はカーネル関数の形のみ(rbf:放射基底関数)を指定している．

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)

Let's use our previously defined function to visualize the fit and identify the support vectors (see the following figure):

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none');

Using this kernelized support vector machine, we learn a suitable nonlinear decision boundary.
This kernel transformation strategy is used often in machine learning to turn fast linear methods into fast nonlinear methods, especially for models in which the kernel trick can be used.

### ソフトマージン

2つのクラスのデータがどうしてもオーバーラップする場合はどうすればよいか？  
例を見てみる．

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

こういうケースでは完全なマージンを見つけることはできないので，ある程度のはみだしを許容する必要がある．このとき，どの程度入り込むことまで許容するか(硬いか，柔らかいか)を定めることができる．  
次の例では，パラメータCが大きいと「硬く」，小さいと「柔らかい」．

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

The optimal value of `C` will depend on your dataset, and you should tune this parameter using cross-validation or a similar procedure (refer back to [Hyperparameters and Model Validation](05.03-Hyperparameters-and-Model-Validation.ipynb)).

## 例：顔の認識

顔画像認識の例  
scikit-learnの*Wild*という顔画像データセットを用いる．これは著名人の顔画像のデータである．  
まずデータを読み込む．

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

ちょっと画像を見てみる．  
各国の首脳や政治家の画像である．

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

各画像は62x47ピクセルである．今回はこのピクセル値をそのままではなく，主成分分析により150個の基本成分に抽出したものを用いる．

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150, whiten=True,
          svd_solver='randomized', random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

例によって訓練データと検証データに分ける．

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

マージンの硬さ(C)と放射基底関数のカーネルサイズ(gamma)をいくつか用意し，最良のものを求める．  
結果としてC=5，gamma=.001の時に最良であることがわかる．  
**数分かかる**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

検証データで検証する．いくつかの検証データのサンプルの分類結果を表示する．名前が赤くなっているものが，間違った分類となる．

In [ ]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

（表示される間違ったデータは実行ごとに異なる）

結果を統計データとして表示する．

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

混同行列にする．

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d',
            cbar=False, cmap='Blues',
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');